Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/machine-learning-pipelines/nyc-taxi-data-regression-model-building/nyc-taxi-data-regression-model-building.png)

# AML Pipelines With Azure Stack K8s Clusters

In this notebook, you learn how to prepare data for regression modeling by using open source library [pandas](https://pandas.pydata.org/). You run various transformations to filter and combine two different NYC taxi datasets. Once you prepare the NYC taxi data for regression modeling, then you will use [Azure Machine Learning Pipelines](https://aka.ms/aml-pipelines) to define your machine learning goals and constraints as well a custom training.  The trained model then registered to AML workspace.

After you complete building the model, you can predict the cost of a taxi trip by training a model on data features. These features include the pickup day and time, the number of passengers, and the pickup location.

## Prerequisite

*     A Kubernetes cluster deployed on Azure Stack Hub, connected to Azure through ARC.
     
   To deploy a Kubernetes cluster with AKS engine on Azure Stack Hub, please see [here](https://docs.microsoft.com/en-us/azure-stack/user/azure-stack-kubernetes-aks-engine-overview?view=azs-2008).


*     Connect  Azure Stack Hub’s Kubernetes cluster to  Azure via [ Azure ARC](https://docs.microsoft.com/en-us/azure/azure-arc/kubernetes/connect-cluster).(For installation, please read note below first)
    
 Important Note: This notebook  requires az extension k8s-extension >= 0.1 and connectedk8s >= 0.3.2 installed on the cluster's master node. Current version for connectedk8s in public preview release via [ Azure ARC](https://docs.microsoft.com/en-us/azure/azure-arc/kubernetes/connect-cluster) is 0.2.8, so you need to install [private preview](https://github.com/Azure/azure-arc-kubernetes-preview/blob/master/docs/k8s-extensions.md). For your convenience, we have include the wheel files, and you just need to run:
 
 <pre>
 az extension add --source connectedk8s-0.3.5-py2.py3-none-any.whl --yes
 az extension add --source k8s_extension-0.1PP.8-py2.py3-none-any.whl --yes
 </pre>
  

*     Setup Azure Machine Learning workspace on Azure.

   please make sure the following 
   [Prerequisites](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-manage-workspace?tabs=python#prerequisites) are met (we recommend using the Python SDK when communicating with Azure Machine Learning so make sure the SDK is properly installed). We strongly recommend learning more about the [innerworkings and concepts in Azure Machine Learning](https://docs.microsoft.com/en-us/azure/machine-learning/concept-azure-machine-learning-architecture) before continuing with the rest of this article (optional)


*      Last but not least, you need to be able to run a Notebook. 

   If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the configuration Notebook located at [here](https://github.com/Azure/MachineLearningNotebooks) first if you haven't. This sets you up with a working config file that has information on your workspace, subscription id, etc.

## Prepare data for regression modeling
First, we will prepare data for regression modeling. We will leverage the convenience of Azure Open Datasets along with the power of Azure Machine Learning service to create a regression model to predict NYC taxi fare prices. Perform `pip install azureml-opendatasets` to get the open dataset package.  The Open Datasets package contains a class representing each data source (NycTlcGreen and NycTlcYellow) to easily filter date parameters before downloading.


### Load data
Begin by creating a dataframe to hold the taxi data. When working in a non-Spark environment, Open Datasets only allows downloading one month of data at a time with certain classes to avoid MemoryError with large datasets. To download a year of taxi data, iteratively fetch one month at a time, and before appending it to green_df_raw, randomly sample 500 records from each month to avoid bloating the dataframe. Then preview the data. To keep this process short, we are sampling data of only 1 month.

Note: Open Datasets has mirroring classes for working in Spark environments where data size and memory aren't a concern.

In [1]:
import azureml.core
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

C:\Users\v-songshanli\Anaconda3\envs\pythonProject\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\v-songshanli\Anaconda3\envs\pythonProject\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\v-songshanli\Anaconda3\envs\pythonProject\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %
Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-core 1.19.0 (c:\users\v-songshanli\anaconda3\envs\pythonproject\lib\site-packages), Requirement.parse('azureml-core~=1.18.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-dataset-runtime 1.19.0.post1 (c:\users\v-so

SDK version: 1.19.0


In [2]:
from azureml.opendatasets import NycTlcGreen, NycTlcYellow
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

green_df_raw = pd.DataFrame([])
start = datetime.strptime("1/1/2016","%m/%d/%Y")
end = datetime.strptime("1/31/2016","%m/%d/%Y")

number_of_months = 1
sample_size = 5000

for sample_month in range(number_of_months):
    temp_df_green = NycTlcGreen(start + relativedelta(months=sample_month), end + relativedelta(months=sample_month)) \
        .to_pandas_dataframe()
    green_df_raw = green_df_raw.append(temp_df_green.sample(sample_size))

[Info] read from C:\Users\V-SONG~1\AppData\Local\Temp\tmpbzns7ma5\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=1\part-00119-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2689-1.c000.snappy.parquet


In [3]:
yellow_df_raw = pd.DataFrame([])
start = datetime.strptime("1/1/2016","%m/%d/%Y")
end = datetime.strptime("1/31/2016","%m/%d/%Y")

sample_size = 500

for sample_month in range(number_of_months):
    temp_df_yellow = NycTlcYellow(start + relativedelta(months=sample_month), end + relativedelta(months=sample_month)) \
        .to_pandas_dataframe()
    yellow_df_raw = yellow_df_raw.append(temp_df_yellow.sample(sample_size))

[Info] read from C:\Users\V-SONG~1\AppData\Local\Temp\tmpi78gp5ql\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\yellow\puYear=2016\puMonth=1\part-00000-tid-8898858832658823408-a1de80bd-eed3-4d11-b9d4-fa74bfbd47bc-426339-90.c000.snappy.parquet
[Info] read from C:\Users\V-SONG~1\AppData\Local\Temp\tmpi78gp5ql\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\yellow\puYear=2016\puMonth=1\part-00001-tid-8898858832658823408-a1de80bd-eed3-4d11-b9d4-fa74bfbd47bc-426336-89.c000.snappy.parquet
[Info] read from C:\Users\V-SONG~1\AppData\Local\Temp\tmpi78gp5ql\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\yellow\puYear=2016\puMonth=1\part-00002-tid-8898858832658823408-a1de80bd-eed3-4d11-b9d4-fa74bfbd47bc-426334-91.c000.snappy.parquet
[Info] read from C:\Users\V-SONG~1\AppData\Local\Temp\tmpi78gp5ql\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\yellow\puYear=2016\puMonth=1\part-00003-tid-8898858832658823408-a1de80bd-eed3-4d11-b9d4-fa74bfbd47bc-426340-87.c000.snappy.parquet
[Inf

### See the data

In [4]:
from IPython.display import display

display(green_df_raw.head(5))
display(yellow_df_raw.head(5))

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType,puYear,puMonth
784738,2,2016-01-11 09:15:02,2016-01-11 09:26:42,1,2.27,None,None,-73.907066,40.765537,-73.937004,...,0.0,0.5,0.3,1.00,0.0,NaN,11.80,1.0,2016,1
1027281,2,2016-01-24 17:30:44,2016-01-24 17:33:07,1,0.49,None,None,-73.972534,40.651367,-73.965607,...,0.0,0.5,0.3,1.44,0.0,NaN,6.24,1.0,2016,1
512847,2,2016-01-11 17:32:18,2016-01-11 17:39:58,1,1.10,None,None,-73.945450,40.807983,-73.960060,...,1.0,0.5,0.3,0.00,0.0,NaN,8.80,1.0,2016,1
382128,2,2016-01-01 10:09:17,2016-01-01 10:25:13,5,7.01,None,None,-73.957565,40.722256,-73.909538,...,0.0,0.5,0.3,0.00,0.0,NaN,21.80,1.0,2016,1
121970,2,2016-01-19 14:15:45,2016-01-19 14:19:49,1,0.94,None,None,-73.871948,40.734188,-73.879570,...,0.0,0.5,0.3,1.89,0.0,NaN,8.19,1.0,2016,1


,vendorID,tpepPickupDateTime,tpepDropoffDateTime,passengerCount,tripDistance,puLocationId,doLocationId,startLon,startLat,endLon,...,paymentType,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,totalAmount,puYear,puMonth
534814,1,2016-01-12 22:02:24,2016-01-12 22:14:51,1,2.60,None,None,-73.985275,40.732029,-73.984634,...,1,11.0,0.5,0.5,0.3,2.45,0.0,14.75,2016,1
430042,2,2016-01-30 19:09:07,2016-01-30 19:20:10,1,1.64,None,None,-73.978630,40.746063,-73.980904,...,2,9.0,0.0,0.5,0.3,0.00,0.0,9.80,2016,1
343241,2,2016-01-21 16:52:20,2016-01-21 16:57:23,1,0.55,None,None,-73.992607,40.730476,-73.990166,...,2,5.0,1.0,0.5,0.3,0.00,0.0,6.80,2016,1
510869,2,2016-01-01 15:06:12,2016-01-01 15:15:43,1,2.39,None,None,-73.979485,40.776581,-73.974937,...,1,9.5,0.0,0.5,0.3,2.06,0.0,12.36,2016,1
391133,2,2016-01-30 17:10:33,2016-01-30 17:33:55,1,2.26,None,None,-73.973297,40.747341,-73.973358,...,1,15.5,0.0,0.5,0.3,3.26,0.0,19.56,2016,1


### Download data locally and then upload to Azure Blob

This is a one-time process to save the data in the default datastore. 

To set up datastore using an azure stack hub storage account, please refer to [Train_azure_arc](https://github.com/Azure/AML-Kubernetes/blob/master/docs/ASH/Train-AzureArc.md#create-and-configure-azure-stack-hubs-storage-account). 

To register the dataset manually, please refer to this [video](https://msit.microsoftstream.com/video/51f7a3ff-0400-b9eb-2703-f1eb38bc6232)

In [5]:
import os
dataDir = "data"

if not os.path.exists(dataDir):
    os.mkdir(dataDir)

greenDir = dataDir + "/green"
yelloDir = dataDir + "/yellow"

if not os.path.exists(greenDir):
    os.mkdir(greenDir)
    
if not os.path.exists(yelloDir):
    os.mkdir(yelloDir)
    
greenTaxiData = greenDir + "/unprepared.parquet"
yellowTaxiData = yelloDir + "/unprepared.parquet"

green_df_raw.to_csv(greenTaxiData, index=False)
yellow_df_raw.to_csv(yellowTaxiData, index=False)

print("Data written to local folder.")

Data written to local folder.


In [6]:
from azureml.core import Workspace, Datastore

ws = Workspace.from_config()
print("Workspace: " + ws.name, "Region: " + ws.location, sep = '\n')

# Default datastore

#datastore_name = "ashstore"
#default_store =  Datastore.get(ws, datastore_name)

default_store = ws.get_default_datastore() 

default_store.upload_files([greenTaxiData], 
                           target_path = 'green', 
                           overwrite = True, 
                           show_progress = True)

default_store.upload_files([yellowTaxiData], 
                           target_path = 'yellow', 
                           overwrite = True, 
                           show_progress = True)

print("Upload calls completed.")

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Workspace: sl-ash2-mal
Region: eastus
Uploading an estimated of 1 files
Uploading data/green/unprepared.parquet
Uploaded data/green/unprepared.parquet, 1 files out of an estimated total of 1
Uploaded 1 files
Uploading an estimated of 1 files
Uploading data/yellow/unprepared.parquet
Uploaded data/yellow/unprepared.parquet, 1 files out of an estimated total of 1
Uploaded 1 files
Upload calls completed.


### Create and register datasets

By creating a dataset, you create a reference to the data source location. If you applied any subsetting transformations to the dataset, they will be stored in the dataset as well. You can learn more about the what subsetting capabilities are supported by referring to [documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabular_dataset.tabulardataset?view=azure-ml-py#remarks). The data remains in its existing location, so no extra storage cost is incurred.

In [7]:
from azureml.core import Dataset
green_taxi_data = Dataset.Tabular.from_delimited_files(default_store.path('green/unprepared.parquet'))
yellow_taxi_data = Dataset.Tabular.from_delimited_files(default_store.path('yellow/unprepared.parquet'))

Register the taxi datasets with the workspace so that you can reuse them in other experiments or share with your colleagues who have access to your workspace.

In [8]:
green_taxi_data = green_taxi_data.register(ws, 'green_taxi_data')
yellow_taxi_data = yellow_taxi_data.register(ws, 'yellow_taxi_data')

### Setup Compute By Attaching a ARC Kubernetes Cluster

The attaching code here depends  python package azureml-contrib-k8s which current is in private preview. Install private preview branch of AzureML SDK by running following command (private preview):

<pre>
pip install --disable-pip-version-check --extra-index-url https://azuremlsdktestpypi.azureedge.net/azureml-contrib-k8s-preview/D58E86006C65 azureml-contrib-k8s
</pre>

In [9]:
from azureml.contrib.core.compute.arckubernetescompute import ArcKubernetesCompute

resource_id = "/subscriptions/6b736da6-3246-44dd-a0b8-b5e95484633d/resourceGroups/AML-stack-val/providers/Microsoft.Kubernetes/connectedClusters/kub-orlando-Test"
attach_name = "peymanarc"
    
attach_config = ArcKubernetesCompute.attach_configuration(resource_id = resource_id)
attach_result = ArcKubernetesCompute.attach(ws ,attach_name, attach_config)
attach_result.wait_for_completion(show_output=True)

print(attach_result)

aml_compute = ws.compute_targets[attach_name]

SucceededProvisioning operation finished, operation "Succeeded"
ArcKubernetesCompute(workspace=Workspace.create(name='sl-ash2-mal', subscription_id='6b736da6-3246-44dd-a0b8-b5e95484633d', resource_group='sl-ash2'), name=peymanarc, id=/subscriptions/6b736da6-3246-44dd-a0b8-b5e95484633d/resourceGroups/sl-ash2/providers/Microsoft.MachineLearningServices/workspaces/sl-ash2-mal/computes/peymanarc, type=ArcKubernetes, provisioning_state=Succeeded, location=eastus, tags=None)


#### Define RunConfig for the compute
We will also use `pandas`, `scikit-learn`,  `pyarrow` for the pipeline steps. Defining the `runconfig` for that.

In [10]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# Create a new runconfig object
aml_run_config = RunConfiguration()

# Use the aml_compute you created above. 
aml_run_config.target = aml_compute

# Enable Docker
aml_run_config.environment.docker.enabled = True

# Use conda_dependencies.yml to create a conda environment in the Docker image for execution
aml_run_config.environment.python.user_managed_dependencies = False

# Specify CondaDependencies obj, add necessary packages
aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['pandas','scikit-learn'], 
    pip_packages=['azureml-sdk', 'pyarrow'])

print ("Run configuration created.")

Run configuration created.


### Prepare data
Now we will prepare for regression modeling by using `pandas`. We run various transformations to filter and combine two different NYC taxi datasets.

We achieve this by creating a separate step for each transformation as this allows us to reuse the steps and saves us from running all over again in case of any change. We will keep data preparation scripts in one subfolder and training scripts in another.

> The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

#### Define Useful Columns
Here we are defining a set of "useful" columns for both Green and Yellow taxi data.

In [11]:
display(green_df_raw.columns)
display(yellow_df_raw.columns)

# useful columns needed for the Azure Machine Learning NYC Taxi tutorial
useful_columns = str(["cost", "distance", "dropoff_datetime", "dropoff_latitude", 
                      "dropoff_longitude", "passengers", "pickup_datetime", 
                      "pickup_latitude", "pickup_longitude", "store_forward", "vendor"]).replace(",", ";")

print("Useful columns defined.")

Index(['vendorID', 'lpepPickupDatetime', 'lpepDropoffDatetime',
       'passengerCount', 'tripDistance', 'puLocationId', 'doLocationId',
       'pickupLongitude', 'pickupLatitude', 'dropoffLongitude',
       'dropoffLatitude', 'rateCodeID', 'storeAndFwdFlag', 'paymentType',
       'fareAmount', 'extra', 'mtaTax', 'improvementSurcharge', 'tipAmount',
       'tollsAmount', 'ehailFee', 'totalAmount', 'tripType', 'puYear',
       'puMonth'],
      dtype='object')

Index(['vendorID', 'tpepPickupDateTime', 'tpepDropoffDateTime',
       'passengerCount', 'tripDistance', 'puLocationId', 'doLocationId',
       'startLon', 'startLat', 'endLon', 'endLat', 'rateCodeId',
       'storeAndFwdFlag', 'paymentType', 'fareAmount', 'extra', 'mtaTax',
       'improvementSurcharge', 'tipAmount', 'tollsAmount', 'totalAmount',
       'puYear', 'puMonth'],
      dtype='object')

Useful columns defined.


#### Cleanse Green taxi data

In [12]:
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep

# python scripts folder
prepare_data_folder = './scripts/prepdata'

# rename columns as per Azure Machine Learning NYC Taxi tutorial

green_columns = {
        "vendorID": "vendor",
        "lpepPickupDatetime": "pickup_datetime",
        "lpepDropoffDatetime": "dropoff_datetime",
        "storeAndFwdFlag": "store_forward",
        "pickupLongitude": "pickup_longitude",
        "pickupLatitude": "pickup_latitude",
        "dropoffLongitude": "dropoff_longitude",
        "dropoffLatitude": "dropoff_latitude",
        "passengerCount": "passengers",
        "fareAmount": "cost",
        "tripDistance": "distance"
    }

green_columns_key = str(list(green_columns.keys())).replace(",", ";")
green_columns_value = str(list(green_columns.values())).replace(",", ";")
    
# Define output after cleansing step
cleansed_green_data = PipelineData("cleansed_green_data", datastore=default_store).as_dataset()

print('Cleanse script is in {}.'.format(os.path.realpath(prepare_data_folder)))

# cleansing step creation
# See the cleanse.py for details about input and output
cleansingStepGreen = PythonScriptStep(
    name="Cleanse Green Taxi Data",
    script_name="cleanse.py", 
    arguments=["--useful_columns", useful_columns,
               "--columns_key", green_columns_key,
                "--columns_value", green_columns_value,
               "--output_cleanse", cleansed_green_data],
    inputs=[green_taxi_data.as_named_input('raw_data')],
    outputs=[cleansed_green_data],
    compute_target=aml_compute,
    runconfig=aml_run_config,
    source_directory=prepare_data_folder,
    allow_reuse=True
)

print("cleansingStepGreen created.")

Cleanse script is in C:\Users\v-songshanli\projects\AML-Kubernetes\docs\ASH\notebooks\pipeline\scripts\prepdata.
cleansingStepGreen created.


#### Cleanse Yellow taxi data

In [13]:
yellow_columns = {
    "vendorID": "vendor",
    "tpepPickupDateTime": "pickup_datetime",
    "tpepDropoffDateTime": "dropoff_datetime",
    "storeAndFwdFlag": "store_forward",
    "startLon": "pickup_longitude",
    "startLat": "pickup_latitude",
    "endLon": "dropoff_longitude",
    "endLat": "dropoff_latitude",
    "passengerCount": "passengers",
    "fareAmount": "cost",
    "tripDistance": "distance"
}

yellow_columns_key = str(list(yellow_columns.keys())).replace(",", ";")
yellow_columns_value = str(list(yellow_columns.values())).replace(",", ";")
    
# Define output after cleansing step
cleansed_yellow_data = PipelineData("cleansed_yellow_data", datastore=default_store).as_dataset()

print('Cleanse script is in {}.'.format(os.path.realpath(prepare_data_folder)))

# cleansing step creation
# See the cleanse.py for details about input and output
cleansingStepYellow = PythonScriptStep(
    name="Cleanse Yellow Taxi Data",
    script_name="cleanse.py", 
    arguments=["--useful_columns", useful_columns,
               "--columns_key", yellow_columns_key,
                "--columns_value", yellow_columns_value,
               "--output_cleanse", cleansed_yellow_data],
    inputs=[yellow_taxi_data.as_named_input('raw_data')],
    outputs=[cleansed_yellow_data],
    compute_target=aml_compute,
    runconfig=aml_run_config,
    source_directory=prepare_data_folder,
    allow_reuse=True
)

print("cleansingStepYellow created.")

Cleanse script is in C:\Users\v-songshanli\projects\AML-Kubernetes\docs\ASH\notebooks\pipeline\scripts\prepdata.
cleansingStepYellow created.


#### Merge cleansed Green and Yellow datasets
We are creating a single data source by merging the cleansed versions of Green and Yellow taxi data.

In [14]:
# Define output after merging step
merged_data = PipelineData("merged_data", datastore=default_store).as_dataset()

print('Merge script is in {}.'.format(os.path.realpath(prepare_data_folder)))

# merging step creation
# See the merge.py for details about input and output
mergingStep = PythonScriptStep(
    name="Merge Taxi Data",
    script_name="merge.py", 
    arguments=["--output_merge", merged_data],
    inputs=[cleansed_green_data.parse_parquet_files(),
            cleansed_yellow_data.parse_parquet_files()],
    outputs=[merged_data],
    compute_target=aml_compute,
    runconfig=aml_run_config,
    source_directory=prepare_data_folder,
    allow_reuse=True
)

print("mergingStep created.")

Merge script is in C:\Users\v-songshanli\projects\AML-Kubernetes\docs\ASH\notebooks\pipeline\scripts\prepdata.
mergingStep created.


#### Filter data
This step filters out coordinates for locations that are outside the city border. We use a TypeConverter object to change the latitude and longitude fields to decimal type. 

In [15]:
# Define output after merging step
filtered_data = PipelineData("filtered_data", datastore=default_store).as_dataset()

print('Filter script is in {}.'.format(os.path.realpath(prepare_data_folder)))

# filter step creation
# See the filter.py for details about input and output
filterStep = PythonScriptStep(
    name="Filter Taxi Data",
    script_name="filter.py", 
    arguments=["--output_filter", filtered_data],
    inputs=[merged_data.parse_parquet_files()],
    outputs=[filtered_data],
    compute_target=aml_compute,
    runconfig = aml_run_config,
    source_directory=prepare_data_folder,
    allow_reuse=True
)

print("FilterStep created.")

Filter script is in C:\Users\v-songshanli\projects\AML-Kubernetes\docs\ASH\notebooks\pipeline\scripts\prepdata.
FilterStep created.


#### Normalize data
In this step, we split the pickup and dropoff datetime values into the respective date and time columns and then we rename the columns to use meaningful names.

In [16]:
# Define output after normalize step
normalized_data = PipelineData("normalized_data", datastore=default_store).as_dataset()

print('Normalize script is in {}.'.format(os.path.realpath(prepare_data_folder)))

# normalize step creation
# See the normalize.py for details about input and output
normalizeStep = PythonScriptStep(
    name="Normalize Taxi Data",
    script_name="normalize.py", 
    arguments=["--output_normalize", normalized_data],
    inputs=[filtered_data.parse_parquet_files()],
    outputs=[normalized_data],
    compute_target=aml_compute,
    runconfig = aml_run_config,
    source_directory=prepare_data_folder,
    allow_reuse=True
)

print("normalizeStep created.")

Normalize script is in C:\Users\v-songshanli\projects\AML-Kubernetes\docs\ASH\notebooks\pipeline\scripts\prepdata.
normalizeStep created.


#### Transform data
Transform the normalized taxi data to final required format. This steps does the following:

- Split the pickup and dropoff date further into the day of the week, day of the month, and month values. 
- To get the day of the week value, uses the derive_column_by_example() function. The function takes an array parameter of example objects that define the input data, and the preferred output. The function automatically determines the preferred transformation. For the pickup and dropoff time columns, split the time into the hour, minute, and second by using the split_column_by_example() function with no example parameter.
- After new features are generated, use the drop_columns() function to delete the original fields as the newly generated features are preferred. 
- Rename the rest of the fields to use meaningful descriptions.

In [17]:
# Define output after transform step
transformed_data = PipelineData("transformed_data", datastore=default_store).as_dataset()

print('Transform script is in {}.'.format(os.path.realpath(prepare_data_folder)))

# transform step creation
# See the transform.py for details about input and output
transformStep = PythonScriptStep(
    name="Transform Taxi Data",
    script_name="transform.py", 
    arguments=["--output_transform", transformed_data],
    inputs=[normalized_data.parse_parquet_files()],
    outputs=[transformed_data],
    compute_target=aml_compute,
    runconfig = aml_run_config,
    source_directory=prepare_data_folder,
    allow_reuse=True
)

print("transformStep created.")

Transform script is in C:\Users\v-songshanli\projects\AML-Kubernetes\docs\ASH\notebooks\pipeline\scripts\prepdata.
transformStep created.


### Split the data into train and test sets
This function segregates the data into dataset for model training and dataset for testing.

In [18]:
train_model_folder = './scripts/trainmodel'

# train and test splits output
output_split_train = PipelineData("output_split_train", datastore=default_store).as_dataset()
output_split_test = PipelineData("output_split_test", datastore=default_store).as_dataset()

print('Data spilt script is in {}.'.format(os.path.realpath(train_model_folder)))

# test train split step creation
# See the train_test_split.py for details about input and output
testTrainSplitStep = PythonScriptStep(
    name="Train Test Data Split",
    script_name="train_test_split.py", 
    arguments=["--output_split_train", output_split_train,
               "--output_split_test", output_split_test],
    inputs=[transformed_data.parse_parquet_files()],
    outputs=[output_split_train, output_split_test],
    compute_target=aml_compute,
    runconfig = aml_run_config,
    source_directory=train_model_folder,
    allow_reuse=True
)

print("testTrainSplitStep created.")

Data spilt script is in C:\Users\v-songshanli\projects\AML-Kubernetes\docs\ASH\notebooks\pipeline\scripts\trainmodel.
testTrainSplitStep created.


#### Create experiment

In [19]:
from azureml.core import Experiment

experiment = Experiment(ws, 'NYCTaxi_Tutorial_Pipelines')

print("Experiment created")

Experiment created


#### Define a custom training step

In [20]:
training_dataset = output_split_train.parse_parquet_files().keep_columns(['pickup_weekday','pickup_hour', 'distance','passengers', 'vendor', 'cost'])
test_dataset = output_split_test.parse_parquet_files().keep_columns(['pickup_weekday', 'pickup_hour', 'distance', 'passengers', 'vendor', 'cost'])

train_step = PythonScriptStep(
        name="train_step",
        script_name="train_step.py",
        arguments=[],
        inputs=[training_dataset, test_dataset],
        outputs=[],
        compute_target=aml_compute,
        runconfig=aml_run_config,
        source_directory=train_model_folder,
        allow_reuse=True
    )

print("train_step created.")

train_step created.


#### Build and run the pipeline, register the model

In [ ]:
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

pipeline_steps = [train_step]

pipeline = Pipeline(workspace = ws, steps=pipeline_steps)
print("Pipeline is built.")

pipeline_run = experiment.submit(pipeline, regenerate_outputs=False)

print("Pipeline submitted for execution.")

pipeline_run.wait_for_completion()

train_step_run = pipeline_run.find_step_run(train_step.name)[0]
train_step_run.register_model(model_name='taxi_model', model_path='outputs/taxi.pkl')

Pipeline is built.
Created step train_step [30b2d9f6][ec310479-8a37-46f8-8f4e-b37873d1af85], (This step is eligible to reuse a previous run's output)
Created step Train Test Data Split [1bf8391e][a0afea19-96b3-41ad-ba42-8e0aada3317a], (This step is eligible to reuse a previous run's output)
Created step Transform Taxi Data [fcd859a3][e3ef1122-4c72-4d83-bdf8-2a10c5d9ca06], (This step is eligible to reuse a previous run's output)
Created step Normalize Taxi Data [b319f48d][17df0506-b087-453b-bcae-f09a91a44676], (This step is eligible to reuse a previous run's output)Created step Filter Taxi Data [e78049b1][593439b5-0c67-4148-a47f-baed6aee4601], (This step is eligible to reuse a previous run's output)
Created step Merge Taxi Data [e3ae2706][f2580a35-e901-41eb-8ce3-85e8bc6ba54a], (This step is eligible to reuse a previous run's output)
Created step Cleanse Green Taxi Data [af4e0449][3f15d738-4d66-4311-bfde-221f259d3f50], (This step is eligible to reuse a previous run's output)
Created step

Expected a StepRun object but received <class 'azureml.core.run.Run'> instead.
This usually indicates a package conflict with one of the dependencies of azureml-core or azureml-pipeline-core.
Please check for package conflicts in your python environment


The machine learning model named "taxi_model" should be registered in your AML workspace.

## Next Steps

1. Learn how to [download model then upload to Azure Storage blobs](../AML-model-download-upload.ipynb)
2. Learn how to [inference using KFServing with model in Azure Storage Blobs](https://aka.ms/kfas)